In [1]:
import pandas as pd
import numpy as np
import pyro
import pyro.distributions as dist
from pyro.nn import PyroModule, PyroSample
import torch.nn as nn
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
import os
import concurrent.futures

# IRI Database Treatment

In [34]:
df = pd.read_excel('c:/Users/sergi/OneDrive/Escritorio/UPV/4_curso/TFG/Data/IRI/IRI_netherlands_v0.xlsx')

# drop columns
df = df.drop(columns = ['BPS1 (created)', 'BPS2 (created)'])
# rename a column
df.rename(columns={'Age of the pavement at the time of measurement':'Age', 'IRI Measurement value':'IRI'}, inplace=True)
# drop nan values
df = df.dropna()

In [36]:
df.head()

,ID lane segment,IRI,Date measurement,Regional service,Road,lane,From,To,Segment length,Segment width,Construction date,Surface layer type,Age,Original version number,BPS3
0,83757,2.1,2018-02-26,WNN,1HRL,2R-L,21.0,21.1,100,3.5,2017-08-01,DAB,0,19,WNN_10_1HRL_2R-L_21.0_21.1
1,84654,2.1,2018-02-26,WNN,1HRL,2R-L,21.0,21.1,100,3.5,2017-08-01,DAB,0,20,WNN_10_1HRL_2R-L_21.0_21.1
2,92691,2.1,2018-02-26,WNN,1HRL,2R-L,21.0,21.1,100,3.5,2017-08-01,DAB,0,21,WNN_10_1HRL_2R-L_21.0_21.1
3,84654,5.3,2019-08-08,WNN,1HRL,2R-L,21.0,21.1,100,3.5,2017-08-01,DAB,2,20,WNN_10_1HRL_2R-L_21.0_21.1
4,92691,5.3,2019-08-08,WNN,1HRL,2R-L,21.0,21.1,100,3.5,2017-08-01,DAB,2,21,WNN_10_1HRL_2R-L_21.0_21.1


### Negative Age Delete

In [37]:
df = df.loc[df['Age']>=0]

### Adition of Construction Year to BPS3

In [39]:
c_year = pd.to_datetime(df['Construction date'], format='%Y%m%d').dt.year.astype(str)

# concatenate BPS3 with c_year column by _

df['BPS3'] = df['BPS3'] + '_' + c_year

In [40]:
df.to_excel('c:/Users/sergi/OneDrive/Escritorio/UPV/4_curso/TFG/Data/IRI/IRI_netherlands_v1.xlsx', index = False)

## BPS3 Duplicates Delete

### Data Import

In [5]:
# import excel file
df = pd.read_excel('c:/Users/sergi/OneDrive/Escritorio/UPV/4_curso/TFG/Data/IRI/IRI_netherlands_v1.xlsx')

In [4]:
df.head()

,Unnamed: 0,ID lane segment,IRI Measurement value,Date measurement,Regional service,Road,lane,From,To,Segment length,Segment width,Construction date,Surface layer type,BPS1 (created),BPS2 (created),Age of the pavement at the time of measurement,Original version number,BPS3
0,70371,83757,2.1,2018-02-26,WNN,1HRL,2R-L,21.0,21.1,100,3.5,2017-08-01,DAB,10_1HRL_2R-L_21.0_21.1,NL_10_1HRL_2R-L_21.0_21.1,0,19,WNN_10_1HRL_2R-L_21.0_21.1
1,99875,84654,2.1,2018-02-26,WNN,1HRL,2R-L,21.0,21.1,100,3.5,2017-08-01,DAB,10_1HRL_2R-L_21.0_21.1,NL_10_1HRL_2R-L_21.0_21.1,0,20,WNN_10_1HRL_2R-L_21.0_21.1
2,102320,92691,2.1,2018-02-26,WNN,1HRL,2R-L,21.0,21.1,100,3.5,2017-08-01,DAB,10_1HRL_2R-L_21.0_21.1,NL_10_1HRL_2R-L_21.0_21.1,0,21,WNN_10_1HRL_2R-L_21.0_21.1
3,99876,84654,5.3,2019-08-08,WNN,1HRL,2R-L,21.0,21.1,100,3.5,2017-08-01,DAB,10_1HRL_2R-L_21.0_21.1,NL_10_1HRL_2R-L_21.0_21.1,2,20,WNN_10_1HRL_2R-L_21.0_21.1
4,102321,92691,5.3,2019-08-08,WNN,1HRL,2R-L,21.0,21.1,100,3.5,2017-08-01,DAB,10_1HRL_2R-L_21.0_21.1,NL_10_1HRL_2R-L_21.0_21.1,2,21,WNN_10_1HRL_2R-L_21.0_21.1


### Process to remove duplicates according to original version number.

We want to have only one record per BPS3 for each year:

1. Iterate over different tuples of (Age,BPS3).
2. Select in each iteration the row that has the key with the highest Original Version Number value. If there are several rows with the same value, select the last one.
3. Save the selected row.

The following code is implemented in a separate python script.

In [ ]:
df = pd.read_excel('c:/Users/sergi/OneDrive/Escritorio/UPV/4_curso/TFG/Data/IRI_netherlands_v1.xlsx')

def delete_BPS3_duplicates(key, age):
    max_version = max(df.loc[(df['BPS3'] == key) & (df['Age'] == age),'Original version number']) #max value of 'original version number' per BPS3 and Age
    x = df.loc[(df['BPS3'] == key) & (df['Age'] == age) & (df['Original version number'] == max_version)].iloc[-1] #search a row based on values of three columns
    return x #we return the last row as there are some duplicates with the same 'original version number'. The last row is for the latest date

futures = []
def main():  
    with concurrent.futures.ProcessPoolExecutor() as executor:
        future_row = {executor.submit(delete_BPS3_duplicates, key, age): 
                        (key,age) for key in df['BPS3'].unique() for age in df.loc[df['BPS3'] == key,'Age'].unique()}
    
        for future in concurrent.futures.as_completed(future_row): 
            futures.append(future.result())
        return futures


if __name__ == '__main__':
    futures = main()
    export_df = pd.DataFrame(futures)
    export_df.to_excel('c:/Users/sergi/OneDrive/Escritorio/UPV/4_curso/TFG/Data/IRI/IRI_netherlands_v2.xlsx', index = False)

# INWEVA Database Treatment

In [ ]:
# open the folder called 'INWEVA' and loop through all the files

path = 'c:/Users/sergi/OneDrive/Escritorio/UPV/4_curso/TFG/Data/INWEVA'
files = os.listdir(path)

inweva = pd.DataFrame()

for file in files:
    print(file)
    df = pd.read_excel(path + '/' + file)
    df = df.loc[(df['Baantype_van']=='HR') & (df['Baantype_naar']=='HR')]
    df['Year'] = int(file[7:11])
    # select columns
    df = df[['Year','Wegnr_van', 'Hm_van', 'Hm_naar', 'Ri_van', 'Ri_naar', 'etmaal_AL', 'etmaal_L1', 'etmaal_L2', 'etmaal_L3']]
    # remove rows that can't be converted to integer in Wegnr_van
    df = df.loc[df['Wegnr_van'].apply(lambda x: type(x)==int)] #there are values in this column with letter on it
    # change type to integer
    for c in df.columns:
        if c != 'Ri_van' and c != 'Ri_naar':
            df[c] = df[c].astype(int)
    df['From'] = round(df['Hm_van']/1000,1) # to match the format of the IRI data
    df['To'] = round(df['Hm_naar']/1000,1)
    inweva = pd.concat([inweva, df], ignore_index=True, axis=0)
# delete nan values
inweva = inweva.dropna()

In [ ]:
inweva.to_excel('c:/Users/sergi/OneDrive/Escritorio/UPV/4_curso/TFG/Data/TreatedData/INWEVA_treated.xlsx', index = False)

## Joining IRI and INWEVA

In [1]:
iri_df = pd.read_excel('c:/Users/sergi/OneDrive/Escritorio/UPV/4_curso/TFG/Data/IRI/IRI_netherlands_v2.xlsx')
traffic_df = pd.read_excel('c:/Users/sergi/OneDrive/Escritorio/UPV/4_curso/TFG/Data/TreatedData/INWEVA_treated.xlsx')

In [8]:

def change_column_types(traffic_df, iri_df):

    # create column called Wegnr_van in iri_df
    iri_df['Wegnr_van'] = iri_df['BPS3'].apply(lambda x: x.split('_')[1])

    # add column Ri_van in iri_df to merge with traffic_df
    iri_df['Ri_van'] = iri_df['Road'].apply(lambda x: x[-1])

    # obtain the year from the IRI dataframe
    iri_df['Year'] = pd.to_datetime(iri_df['Date measurement'], format='%Y%m%d').dt.year
    iri_df['Year'] = iri_df['Year'].astype(int)

    #change the columns From, Year, Wegnr_van and Ri_van to the same type on both dataframes
    iri_df['From'] = iri_df['From'].astype(float)
    traffic_df['From'] = traffic_df['From'].astype(float)
    traffic_df['Year'] = traffic_df['Year'].astype(int)
    iri_df['Wegnr_van'] = iri_df['Wegnr_van'].astype(int)
    traffic_df['Wegnr_van'] = traffic_df['Wegnr_van'].astype(int)
    iri_df['Ri_van'] = iri_df['Ri_van'].astype(str)
    traffic_df['Ri_van'] = traffic_df['Ri_van'].astype(str)
    
    # sort both dataframes by the columns From, Year, Wegnr_van and Ri_van
    iri_df = iri_df.sort_values(by=['From', 'Year', 'Wegnr_van', 'Ri_van'])
    traffic_df = traffic_df.sort_values(by=['From', 'Year', 'Wegnr_van', 'Ri_van'])

    return traffic_df, iri_df

# Function to find overlapping ranges efficiently using vectorized operations
def merge_overlapping_ranges(traffic_df, iri_df):
    # Cartesian product to combine all possible pairs of rows
    cart_product = pd.merge(iri_df, traffic_df, on=['Year','Wegnr_van', 'Ri_van'])

    # Filtering overlapping ranges
    overlapping = (
        (cart_product['From_x'] <= cart_product['To_y']) &
        (cart_product['To_x'] >= cart_product['From_y'])
    )

    # Selecting overlapping rows and creating the merged DataFrame
    merged_df = cart_product[overlapping].assign(
        From=lambda x: np.maximum(x['From_x'], x['From_y']),
        To=lambda x: np.minimum(x['To_x'], x['To_y'])
    ).drop(columns=['From_x', 'From_y', 'To_x', 'To_y'])

    return merged_df

In [9]:

# change the type of the columns
traffic_df, iri_df = change_column_types(traffic_df, iri_df)

# Merge overlapping ranges efficiently
merged_data = merge_overlapping_ranges(traffic_df, iri_df)

In [10]:

# drop the columns that are not needed
merged_data = merged_data.drop(columns=['Ri_naar', 'ID lane segment ', 'Hm_van', 'Hm_naar'])
# rename the columns
merged_data.rename(columns={'Ri_van':'Side', 'Wegnr_van':'Segment ID', 'Regional service':'Region', 'lane':'Lane'}, inplace=True)
# change the order of the columns
merged_data = merged_data[['BPS3', 'Segment ID', 'Construction date', 'Date measurement', 'Year', 'Age', 'Region',
                            'Road', 'Surface layer type', 'Side', 'Lane', 'Segment length', 'Segment width', 'From', 'To',
                              'etmaal_AL', 'etmaal_L1', 'etmaal_L2', 'etmaal_L3', 'IRI']]
# remove empty values
merged_data = merged_data.dropna()

In [11]:

# export the dataframe
merged_data.to_excel('c:/Users/sergi/OneDrive/Escritorio/UPV/4_curso/TFG/Data/TreatedData/IRI_INWEVA.xlsx', index = False)

# KNMI Database Treatment

In [7]:
df = pd.read_csv('C:/Users/sergi/OneDrive/Escritorio/UPV/4_curso/TFG/Data/KNMI/MN-FL/etmgeg_269.txt', skiprows=50)

def KNMI_treatment(df):

    # delete nan values
    df = df.dropna()
    # convert YYYYMMDD to datetime
    df['YYYYMMDD'] = pd.to_datetime(df['YYYYMMDD'], format='%Y%m%d')
    # obtain the year
    df['Year'] = df['YYYYMMDD'].dt.year
    # select a range from the year column
    df = df.loc[df['Year'].between(2013, 2022)]
    # remove spaces from the columns values
    df.columns = df.columns.str.strip()
    # select columns
    df = df[['Year', 'FG', 'TG', 'TN', 'TX', 'RH', 'SQ', 'SP', 'UG']]

    #{'FG':'Wind speed', 'TG':'Temperature', 'TN':'Min temperature', 'TX':'Max temperature', 'RH':'Precipitation Amount',
    #'SQ':'Sunshine duration', 'SP':'Sunshine percentage', 'UG':'Humidity'}
            
    # change all data types to float
    df = df.astype(float)
    df['Year'] = df['Year'].astype(int)
    
    for y in df['Year'].unique():

    # calculate the freeze index
        df.loc[df['Year'] == y, 'FRZ'] = -df.loc[(df['Year'] == y) & (df['TG'] < 0), 'TG'].sum()

    # caculate the freeze-thaw index
        df.loc[df['Year'] == y, 'FRZT'] = df.loc[(df['Year'] == y) & (df['TN'] < 0) & (df['TX'] > 0)].shape[0]

    # calculate the days above 32ºC
        df.loc[df['Year'] == y, 'HOT'] = df.loc[(df['Year'] == y) & (df['TX'] > 320)].shape[0] #the temperature is in tenths of degrees Celsius
    
    # calculate the days below 0ºC
        df.loc[df['Year'] == y, 'COLD'] = df.loc[(df['Year'] == y) & (df['TN'] < 0)].shape[0]
    
    # calculate acumulated precipitation
        df.loc[df['Year'] == y, 'RHA'] = df.loc[(df['Year'] == y) & (df['RH'] > 0), 'RH'].sum()
        
    return df

### Implementation:

In [7]:
# turn off warnings

import warnings

warnings.filterwarnings('ignore')

# iterate through all the subfolders and files

import os
import pandas as pd

path = 'c:/Users/sergi/OneDrive/Escritorio/UPV/4_curso/TFG/Data/KNMI'
folders = os.listdir(path)

knmi = pd.DataFrame()

for folder in folders:
    files = os.listdir(path + '/' + folder)
    for file in files:
        print(file)
        df = pd.read_csv(path + '/' + folder + '/' + file, skiprows=50)
        df = KNMI_treatment(df)
        # select from folder the letter before the dash symbol
        df['Region'] = folder.split('-')[0]
        # add it to he knmi dataframe
        knmi = pd.concat([knmi, df], ignore_index=True, axis=0)

# calculate the mean by year
knmi = knmi.groupby(['Region','Year']).mean().reset_index()
# add it to he knmi dataframe

etmgeg_269.txt
etmgeg_260.txt
etmgeg_279.txt
etmgeg_270.txt
etmgeg_280.txt
etmgeg_275.txt
etmgeg_290.txt
etmgeg_240.txt
etmgeg_344.txt
etmgeg_310.txt
etmgeg_380.txt
etmgeg_391.txt
etmgeg_370.txt


In [8]:
KNMI_treatment(df)

,Year,FG,TG,TN,TX,RH,SQ,SP,UG,FRZ,FRZT,HOT,COLD,RHA
8401,2013,49.0,67.0,32.0,89.0,88.0,21.0,27.0,89.0,839.0,58.0,1.0,73.0,7779.0
8402,2013,52.0,67.0,49.0,83.0,3.0,37.0,48.0,89.0,839.0,58.0,1.0,73.0,7779.0
8403,2013,58.0,98.0,74.0,113.0,9.0,9.0,12.0,94.0,839.0,58.0,1.0,73.0,7779.0
8404,2013,54.0,90.0,81.0,100.0,1.0,0.0,0.0,95.0,839.0,58.0,1.0,73.0,7779.0
8405,2013,32.0,79.0,71.0,85.0,1.0,0.0,0.0,95.0,839.0,58.0,1.0,73.0,7779.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12048,2022,63.0,53.0,23.0,69.0,6.0,47.0,61.0,88.0,197.0,39.0,2.0,41.0,7069.0
12049,2022,98.0,89.0,59.0,109.0,44.0,0.0,0.0,92.0,197.0,39.0,2.0,41.0,7069.0
12050,2022,93.0,89.0,60.0,114.0,28.0,17.0,22.0,86.0,197.0,39.0,2.0,41.0,7069.0
12051,2022,82.0,81.0,45.0,124.0,34.0,0.0,0.0,89.0,197.0,39.0,2.0,41.0,7069.0


In [10]:
# export to excel
knmi.to_excel('c:/Users/sergi/OneDrive/Escritorio/UPV/4_curso/TFG/Data/TreatedData/KNMI_treated.xlsx', index = False)

## Joining the two databases

In [12]:
iri_inweva = pd.read_excel('c:/Users/sergi/OneDrive/Escritorio/UPV/4_curso/TFG/Data/TreatedData/IRI_INWEVA.xlsx')
knmi = pd.read_excel('c:/Users/sergi/OneDrive/Escritorio/UPV/4_curso/TFG/Data/TreatedData/KNMI_treated.xlsx')

In [13]:
# merge both databases by year and region
merged_data = pd.merge(iri_inweva, knmi, on=['Year', 'Region'], how='left')
merged_data.head()

,BPS3,Lane ID,Original version number,Construction date,Date measurement,Year,Age,Region,Road,Surface layer type,...,TX,RH,SQ,SP,UG,FRZ,FRZT,HOT,COLD,RHA
0,WNN_200_1HRR_2R-R_0.4_0.5_2011,200,17,2011-11-30,2015-10-23,2015,3,WNN,1HRR,DAB,...,147.019178,24.153425,51.758904,40.252055,80.646575,42.0,35.0,1.0,35.0,8851.0
1,WNN_200_1HRR_2R-R_0.5_0.6_2006,200,17,2006-12-31,2015-10-23,2015,8,WNN,1HRR,ZOAB,...,147.019178,24.153425,51.758904,40.252055,80.646575,42.0,35.0,1.0,35.0,8851.0
2,WNN_200_1HRR_2R-R_0.6_0.7_2006,200,17,2006-12-31,2015-10-23,2015,8,WNN,1HRR,ZOAB,...,147.019178,24.153425,51.758904,40.252055,80.646575,42.0,35.0,1.0,35.0,8851.0
3,WNN_200_1HRR_2R-R_0.7_0.8_2006,200,17,2006-12-31,2015-10-23,2015,8,WNN,1HRR,ZOAB,...,147.019178,24.153425,51.758904,40.252055,80.646575,42.0,35.0,1.0,35.0,8851.0
4,WNN_200_1HRR_2R-R_0.8_0.9_2006,200,17,2006-12-31,2015-10-23,2015,8,WNN,1HRR,ZOAB,...,147.019178,24.153425,51.758904,40.252055,80.646575,42.0,35.0,1.0,35.0,8851.0


In [14]:
# place IRI column at the end of the dataframe

cols = list(merged_data.columns.values)
cols.pop(cols.index('IRI'))
merged_data = merged_data[cols+['IRI']]

# export to excel
merged_data.to_excel('c:/Users/sergi/OneDrive/Escritorio/UPV/4_curso/TFG/Data/TreatedData/IRI_Completed.xlsx', index = False)